# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [1]:
# imports
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import f1_score
from IPython.display import display


In [2]:
!git clone https://github.com/kfirbar/nlp-course.git

fatal: destination path 'nlp-course' already exists and is not an empty directory.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [3]:

!ls nlp-course/lm-languages-data-new


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
data_files = {'en_df': 'en.csv',
              'es_df': 'es.csv',
              'fr_df': 'fr.csv',
              'in_df': 'in.csv',
              'it_df': 'it.csv',
              'nl_df': 'nl.csv',
              'pt_df': 'pt.csv',
              'tl_df': 'tl.csv'}

    
directory = 'nlp-course/lm-languages-data-new/'    
for (key, value) in data_files.items():
    data_files[key] = directory + value
    
languages_list = list(data_files.keys())
start_token = '↠'
end_token = '↞'

**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [5]:
def preprocess():
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """  
    tokens = []
    for path in data_files.values():
        df = pd.read_csv(path)
        for text in df['tweet_text'].values:
            tokens.extend(list(text))
    return list(set(tokens))

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [6]:
#helper functions
def tweets_to_text(data_file_path, n):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """
    df = pd.read_csv(r''+ data_file_path)
    tweets_list = df['tweet_text'].apply(lambda x: start_token + x + end_token).values
    text = ''.join(tweets_list)
    
    text = start_token * (n-1) + text + end_token * (n-1)

    return text

def reorder_list(List, index_list):
    return [List[i] for i in index_list]

In [7]:
def lm(n, vocabulary, data_file_path, add_one):
    """
    input 
        # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
        # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
        # data_file_path - the data_file from which we record probabilities for our model
        # add_one - True/False (use add_one smoothing or not)
    output:
        model of language base the text and n-gram defenition
    """
    lm_dict = {}
    V = len(vocabulary)

    text = tweets_to_text(data_file_path, n)

    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]

    lm_dict = defaultdict(lambda: defaultdict(lambda: 0))

    for i_n_gram in n_gram:
        n_1_gram = i_n_gram[0:n-1]
        lm_dict[n_1_gram][i_n_gram[n-1]] += 1
    
    for key in lm_dict.keys():
        key_count = sum(lm_dict[key].values())
        inner_dict = {}
        for key_1 in lm_dict[key].keys():
            if add_one:
                inner_dict[key_1] = (lm_dict[key][key_1] + 1) / (key_count + V)
            else:
                inner_dict[key_1] = lm_dict[key][key_1]/ key_count
        if add_one:
            lm_dict[key] = defaultdict(lambda: 1 / (key_count + V), inner_dict)
        else:
            lm_dict[key] = defaultdict(lambda: 0, inner_dict)
            
    return lm_dict

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.


### entropy
- W - all posible token in the language Text
- N - is the numnber of all tokens
$$H(Text) =  -\frac{1}{N}({\sum_{w_i \in W}{P(w_i)log_2(P(w_i))}})$$
- under the assumption the $w_i$ is in the language, we can remove $P(w_i)$
$$H(Text) =  -\frac{1}{N}(\sum_{w_i \in W}log_2(P(w_i)))$$
### perplexity = $2^{H(Text)}$

In [8]:
def eval_tweet(n, N, model, tweet):
  """
  input:
      n  - the n of n-gram
      N - size of tweet
      model - the model of langues
      tweet - text
      
  output:
      probabilities list
  """
  missing_value = 1e-8
  tweet_probabilities = []
  
  for i in range(N - n):
    i_n_gram = tweet[i: i + n]
    key = i_n_gram[0:n-1]
    key_1 = i_n_gram[n-1]

    if key in model:
      if key_1 in model[key]:
        tweet_probabilities.append(model[key][key_1])
      else:
        tweet_probabilities.append(missing_value)
    else:
      tweet_probabilities.append(missing_value)
  
  return tweet_probabilities

In [9]:
def eval(n, model, data_file):
  """
  # input:
      # n - the n-gram that you used to build your model (must be the same number)
      # model - the dictionary (model) to use for calculating perplexity
      # data_file - the tweets file that you wish to calculate a perplexity score for
  # output
      # perlplexity
  """
  df = pd.read_csv(data_file)
  probabilities = []

  for tweet in df['tweet_text'].values:
    tweet = start_token + tweet + end_token
    N = len(tweet)
    tweet_probabilities = eval_tweet(n, N, model, tweet)
    probabilities.extend(tweet_probabilities)
    
  entropy = -math.log2(np.mean(probabilities))
      
  return 2 ** entropy

In [10]:
vocabulary = preprocess()
n = 2
lm_nodel = lm(n, vocabulary, data_files['en_df'], False)
eval(n,lm_nodel, data_files['en_df'])


8.548283124700765

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [11]:
def match(n, add_one, data_files):
    """
    input 
        # n - the n-gram to use for creating n-gram models
        # add_one - use add_one smoothing or not
    output
        # data frame of matching between model to languages
    """
    result_dict = {}
    vocabulary = preprocess()
    for i_language_model in languages_list:
        
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)
        result_dict[i_language_model] = {}

        for i_language_test in languages_list:
            i_language_model_i_score = eval(n, i_model, data_files[i_language_test])
            result_dict[i_language_model][i_language_test] = i_language_model_i_score
    perlexity_df = pd.DataFrame(result_dict)
    return perlexity_df  

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [12]:
 
def run_match(data_files):
    """
    run matching on n-gram model from 1-4, with and without add one
    """
    for n in range(1,5):
        add_one = True
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)

        add_one = False
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)



### run the model matching

In [13]:
model_dict = run_match(data_files)


n = 1, add_one = True


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,22.123312,21.734950,21.950406,23.378052,22.626140,22.590042,21.898668,24.073673
es_df,21.719929,20.354714,20.987790,22.129134,21.467589,21.883629,20.537156,22.820330
fr_df,21.959090,21.010617,21.020677,23.121506,22.109208,21.977180,21.269996,24.125237
in_df,23.336263,22.104845,23.071048,21.130761,22.944087,23.682069,22.111388,22.145444
it_df,22.598276,21.455980,22.073247,22.956870,21.951580,22.842771,21.611530,23.615944
nl_df,22.582913,21.891854,21.961558,23.716990,22.863720,21.925363,22.260596,24.737168
pt_df,21.812619,20.470597,21.178032,22.063952,21.553151,22.180125,20.410431,22.749836
tl_df,24.000485,22.766679,24.042383,22.117647,23.573171,24.669751,22.770133,22.617813


n = 1, add_one = False


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,22.077835,21.687826,21.906717,23.323237,22.575118,22.542475,21.839626,24.012333
es_df,21.675269,20.310549,20.945995,22.077205,21.419149,21.837532,20.481743,22.762143
fr_df,21.913944,20.965044,20.978818,23.067283,22.059336,21.930889,21.212628,24.063765
in_df,23.288324,22.056929,23.025158,21.081154,22.892357,23.632234,22.051779,22.088956
it_df,22.551833,21.409453,22.029316,22.903029,21.902061,22.794679,21.553251,23.555753
nl_df,22.536503,21.844394,21.917847,23.661391,22.812169,21.879180,22.200591,24.674162
pt_df,21.767768,20.426179,21.135860,22.012173,21.504516,22.133409,20.355356,22.691825
tl_df,23.951200,22.717347,23.994586,22.065751,23.520042,24.617867,22.708772,22.560137


n = 2, add_one = True


...

,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,3.836993,5.448475,5.174516,5.447227,5.357483,5.129663,5.399960,5.033745
es_df,5.655030,3.838306,5.107682,5.896175,5.083818,5.665561,4.762284,5.585688
fr_df,5.329020,5.120737,3.721062,5.936231,5.306061,5.245555,5.394554,5.759871
in_df,6.261966,6.203370,6.287293,4.314009,6.251016,6.140492,6.315875,5.500943
it_df,5.833125,5.194200,5.505782,5.915753,3.900101,6.141699,5.399402,5.663712
nl_df,5.313499,5.565887,5.274628,5.606316,5.812449,3.929465,5.767885,5.626244
pt_df,5.762523,4.767150,5.468993,5.996876,5.121950,5.935856,3.632357,5.660587
tl_df,5.802303,6.123357,6.285088,5.490860,5.630289,5.876119,5.932141,3.689011


n = 4, add_one = True


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,10.010735,15.642884,14.239638,16.221259,15.804670,14.062471,17.231398,16.197769
es_df,16.707072,10.389570,13.733327,18.662245,15.282552,16.072010,14.572357,19.419691
fr_df,14.974018,13.984516,9.746510,17.860325,15.770387,14.849662,16.475436,18.714465
in_df,17.259397,18.270578,17.746156,13.185725,18.526299,17.238728,20.234418,17.775232
it_df,17.106526,15.642430,16.115661,19.102400,11.108885,17.656577,18.090532,19.633355
nl_df,12.963716,14.032727,13.075157,14.962729,14.834066,9.438730,15.964931,15.899283
pt_df,17.234505,13.476798,15.312796,19.258158,16.260243,17.471219,11.957385,19.741232
tl_df,17.364396,19.346387,18.871408,17.551899,18.722262,18.030379,20.811830,11.751323


n = 4, add_one = False


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,2.311634,4.785236,4.420210,4.514500,4.524131,4.299011,4.796943,3.947570
es_df,5.418622,2.453607,4.747055,5.804618,4.547829,5.411736,4.214865,5.174293
fr_df,4.925876,4.743356,2.316693,5.612114,4.891423,4.868744,5.157421,5.422332
in_df,6.035627,6.188046,6.075496,2.552894,6.138648,5.933536,6.423965,5.077799
it_df,5.355397,4.614640,4.997058,5.687506,2.413865,5.578548,4.870681,5.083958
nl_df,4.799961,5.205171,4.890692,5.263557,5.341671,2.343462,5.483043,5.231062
pt_df,5.537301,4.132646,5.096857,5.915031,4.566026,5.785380,2.308943,5.189995
tl_df,4.929466,5.122785,5.274965,4.682146,4.760115,5.255277,4.947019,2.244014


**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [14]:
def match_test(n, data_file_path, add_one):
    """
    input
        # n - the n-gram to use for creating n-gram models
        # add_one - use add_one smoothing or not
        # data_file_path - file path
    output
       #  perplexity_df - language model per perplexity score per senetence 
    """
    senstences_list = pd.read_csv(data_file_path)['tweet_text'].to_list()

    result_dict = {}

    for i_language_model in languages_list:
        result_dict[i_language_model] = {}
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)

        for i_test_senstence_idx in range(len(senstences_list)):
            i_test_senstence = senstences_list[i_test_senstence_idx]

            sentence_probabilities = eval_tweet(n, len(i_test_senstence), i_model, i_test_senstence)
            entropy = -math.log2(np.mean(sentence_probabilities))
            i_sentence_model_i_score = 2 ** entropy

            result_dict[i_language_model][i_test_senstence_idx] = i_sentence_model_i_score

    perlexity_df = pd.DataFrame(result_dict)
    perlexity_array = perlexity_df.to_numpy()
    language_match_index = np.argmin(perlexity_array, axis=1)
    language_match_list = reorder_list(languages_list, language_match_index)
    perlexity_df['predict'] = language_match_index
    perlexity_df['predict_language'] = [language_match_list[i][0:2] for i in range(len(language_match_list))]
    display(perlexity_df)

    return perlexity_df

In [15]:
def classify(n, data_file_path, add_one):
    match_dict  = match_test(n, data_file_path, add_one)
    return match_dict

we selected the following parameters to demonstrate classifier function usage:
- n = 3
- add_one = False

In [16]:
n = 3
test_path = directory + 'test.csv'
clasification_result = classify(n, test_path, False)

,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df,predict,predict_language
0,5.024710,7.261914,7.417512,8.270493,7.080208,7.319679,7.319807,6.554967,0,en
1,10.969120,9.098660,11.750203,11.225470,5.645730,14.857745,10.604099,9.518842,4,it
2,10.181503,10.760286,9.881795,9.406162,9.523497,11.450136,10.984212,7.915273,7,tl
3,5.202221,5.386412,5.102922,5.964744,5.285177,4.302613,5.164057,5.878713,5,nl
4,4.573049,4.353399,4.748034,3.943731,4.540290,4.537305,4.591514,3.498314,7,tl
...,...,...,...,...,...,...,...,...,...,...
7994,3.755475,3.052853,3.238446,3.909940,3.368188,3.516530,3.260865,3.951106,1,es
7995,9.648014,8.811993,10.193168,5.102764,9.232547,8.928399,8.757660,6.802859,3,in
7996,15.768045,10.747700,12.923448,18.228846,4.550371,20.984417,10.378884,9.915535,4,it
7997,9.460773,5.493746,6.313035,10.509840,9.090513,7.953040,5.505484,10.986596,1,es


In [17]:
y_true = pd.read_csv(test_path).get('label').to_list()
y_true = list(map(lambda x: languages_list.index(x+'_df'),y_true))
y_pred = clasification_result['predict'].to_list()

**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [18]:
def calc_f1(y_true,y_pred ):
    return np.round(f1_score(y_true, y_pred,average="micro"),3)
f_score_result = calc_f1(y_true,y_pred)
print('The F-score we acheive is ' + str(f_score_result)+'\n')

The F-score we acheive is 0.897



# **Good luck!**